In [1]:
'''
Messy reformation
'''
import json
import re

mydict = {}
mylist = []
mystr = ""
mybool = False

def messyReform(data):
    tlist = []
    if type(data) == type(mylist):
        for element in data:
            newdict = {}
            newdict['label'] = 'dummy'
            newdict['type'] = 'dummy'
            newdict['children'] = messyReform(element)
            tlist.append(newdict)
    else:
        for x in data.keys():
            if type(data[x]) != type(mylist) and type(data[x]) != type(mydict):
                newdict = {}
                newdict['label'] = data[x]
                newdict['type'] = x
                newdict['children'] = []
                tlist.append(newdict)
            else:
                newdict = {}
                newdict['label'] = x
                newdict['type'] = x
                newdict['children'] = messyReform(data[x])
                tlist.append(newdict)
    return tlist

In [4]:
'''
Main part
Messy reformation
'''

import os
import shutil

now = ''
nowtree=''
flag=1
if flag==1:
    now='\\After\\Temp2\\'
    nowtree='after_tree'
else:
    now='\\Before\\Temp2\\'
    nowtree='before_tree'
base_path = 'D:\\Thesis\\All Codes'+now
#directories = [files for root, dirs, files in os.walk(base_path)]
directories = [x[1]
               for x in os.walk(base_path)]

for x in directories[0]:
    print(x)
    filelist = [files for root, dirs, files in os.walk(base_path+x)]
    for y in filelist[0]:
        if 'Parsed' in y and '_Parsed' not in y:
            if(os.stat(base_path+x+"\\"+y).st_size == 0):
                print(x)
                shutil.rmtree(base_path+x)
                break
            with open(base_path+x+"\\"+y) as f:
                data = json.load(f)
            newdict = {}
            newdict['label'] = "root"
            newdict['type'] = "root"
            newdict['children'] = messyReform(data["sqlscript"]["statement"])
            with open(base_path+x+"\\_"+y, 'w') as f:
                json.dump(newdict, f)
                
    

after_1
after_10
after_100
after_101
after_102
after_103
after_104
after_105
after_106
after_107
after_108
after_109
after_11
after_110
after_111
after_112
after_113
after_114
after_115
after_116
after_117
after_118
after_119
after_12
after_120
after_121
after_122
after_123
after_124
after_125
after_126
after_127
after_128
after_129
after_13
after_130
after_131
after_132
after_133
after_134
after_135
after_136
after_137
after_138
after_139
after_14
after_140
after_141
after_142
after_143
after_144
after_145
after_146
after_147
after_148
after_149
after_15
after_150
after_151
after_152
after_153
after_154
after_155
after_156
after_157
after_158
after_159
after_16
after_160
after_161
after_162
after_163
after_164
after_165
after_166
after_167
after_168
after_169
after_17
after_170
after_171
after_172
after_173
after_174
after_175
after_176
after_177
after_178
after_179
after_18
after_180
after_181
after_182
after_183
after_184
after_185
after_186
after_187
after_188
after_189
after_19
af

In [5]:


'''
Travarce the JSON and Return path to the SQL Query String
children - a list
repeat - is multiple Query in one file then 
        repeat = 0 will find first ocurance, 
        repeat = 1 will find second ocurance
'''

mystr = ""

def find_string(children):
    path = []           # contains path to the query string
    # same shape as "path", continuous 1 if connected to ocurance of query, 0 otherwise
    marker = []

    if len(children) == 0:
        return False, path, marker

    for i in range(len(children)):
        # print(children[i])
        if 'label' not in children[i].keys():
            continue
        if type(children[i]['label']) == type(mystr) and re.search(r'DELETE.*FROM.*|SELECT.*FROM.*|INSERT.*INTO.*|UPDATE.*SET.*', children[i]['label'], re.IGNORECASE) != None:
            path.append(i)
            marker.append(1)
            return True, path, marker
        found, nextpath, nextmarker = find_string(children[i]['children'])
        if found == True:
            path.append(i)
            if children[i]['label'] == "PLUS" and nextmarker[0] == 1:
                marker.append(1)
            else:
                marker.append(0)
            path = path + nextpath
            marker = marker + nextmarker
            return True, path, marker

    return False, path, marker


'''
Travarce the children and replace "sqldata" in place using marker
'''


def travarce_replace(children, sqldata, path, marker, pos):
    if pos >= len(marker):
        return children
    if marker[pos] == 0:
        children[path[pos]]['children'] = travarce_replace(
            children[path[pos]]['children'], sqldata, path, marker, pos+1)
    else:
        children[path[pos]] = sqldata

    return children


In [13]:
'''
Main part of Replacing SQL parsed datat in Edit Tree
Change 'Before' 'After' to parse them
'''

import os
import json
import re

now = ''
nowtree=''
flag=1
if flag==1:
    now='\\After\\Temp2\\'
    nowtree='after_tree'
else:
    now='\\Before\\Temp2\\'
    nowtree='before_tree'
base_path = 'D:\\Thesis\\All Codes'
#directories = [files for root, dirs, files in os.walk(base_path)]
directories = [x[1]
               for x in os.walk(base_path+now)]

#print(directories[0])

for x in directories[0]:
    #fileList = []
    try:
        n = int(x.split("_")[1])
        with open(base_path+"\\Edit Trees\\change_pair"+ str(n-1) +".json") as f:
            data = json.load(f)
    except:
        continue
    children = data[nowtree]['children']
    filelist = [files for root, dirs, files in os.walk(base_path+now+x)]
    for y in filelist[0]:
        
        if '_Parsed' in y and os.stat(base_path+now+x+"\\"+y).st_size > 0:
            #fileList.append(y)
            print(x, y)
            with open(base_path+now+x+"\\"+y) as r:
                sqldata = json.load(r)
                
            _, path, marker = find_string(children)
            children = travarce_replace(children, sqldata, path, marker, 0)
            
            with open(base_path+"\\Edit Trees\\change_pair"+ str(n-1) +".json", 'w') as outfile:
                json.dump(data, outfile)
    #print(n, x, fileList)
    
  
'''
i = 0
for x in directoryList:
    with open(x) as r:
        sqldata = json.load(r)
    _ , path, marker, _ = find_string(children, i++)
    children = find_replace(children, sqldata, path, marker, 0)
'''

after_1 _Parsed1.json
after_1 _Parsed2.json
after_1 _Parsed3.json
after_10 _Parsed1.json
after_100 _Parsed1.json
after_101 _Parsed1.json
after_101 _Parsed2.json
after_102 _Parsed1.json
after_103 _Parsed1.json
after_104 _Parsed1.json
after_105 _Parsed1.json
after_106 _Parsed1.json
after_107 _Parsed1.json
after_108 _Parsed1.json
after_108 _Parsed2.json
after_109 _Parsed1.json
after_11 _Parsed1.json
after_110 _Parsed1.json
after_111 _Parsed1.json
after_112 _Parsed1.json
after_113 _Parsed1.json
after_114 _Parsed1.json
after_115 _Parsed1.json
after_117 _Parsed1.json
after_118 _Parsed1.json
after_119 _Parsed1.json
after_12 _Parsed1.json
after_122 _Parsed1.json
after_123 _Parsed1.json
after_126 _Parsed1.json
after_127 _Parsed1.json
after_128 _Parsed1.json
after_13 _Parsed1.json
after_130 _Parsed1.json
after_131 _Parsed1.json
after_132 _Parsed1.json
after_133 _Parsed1.json
after_134 _Parsed1.json
after_135 _Parsed1.json
after_136 _Parsed1.json
after_137 _Parsed1.json
after_138 _Parsed1.json
af

'\ni = 0\nfor x in directoryList:\n    with open(x) as r:\n        sqldata = json.load(r)\n    _ , path, marker, _ = find_string(children, i++)\n    children = find_replace(children, sqldata, path, marker, 0)\n'